# ===============================
# Equity Forecasting Project Blueprint
# Portfolio-Ready Notebook
# ===============================

# Notebook metadata
# Purpose: Showcase overall flow of Equity Forecasting pipeline, sentiment, and forecast
# Modules Highlighted: Dashboard, Market Sentiment (Feeds → Preprocess → Extractor → SentimentModel), Forecasting (LSTM), Combined Output

# Mini NLP Pipeline Illustration
### Sentiment Analysis Pipeline

# 1. **Preprocessing:** Clean text, remove stopwords, normalize.
# 2. **Extraction:** Extract financial-relevant phrases from text.
# 3. **SentimentModel:** 
    - **TextBlob**: Polarity [-1,1], label {positive, neutral, negative}  
    - **FinBERT**: Transformer-based sentiment scoring specific to finance
# 4. **Aggregation:** Average sentiment scores per feed → overall equity sentiment




In [66]:

# -------------------------------
# Imports
# -------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from graphviz import Digraph
import ipywidgets as widgets
from IPython.display import display, Markdown


In [ ]:

# -------------------------------
# Interactive Equity Selector
# -------------------------------
equity_selector = widgets.Dropdown(
    options=['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'RELIANCE.NS', 'TCS.NS'],
    description='Select Equity:',
    disabled=False,
)
display(Markdown("### Select Equity to Preview Sentiment & Forecast"))
display(equity_selector)


### Select Equity to Preview Sentiment & Forecast

Dropdown(description='Select Equity:', options=('AAPL', 'MSFT', 'GOOGL', 'TSLA', 'RELIANCE.NS', 'TCS.NS'), val…

## Sentiment Analysis for MSFT

**Overall Sentiment:** 0.328

## Forecast for MSFT

...

## Sentiment Analysis for AAPL

**Overall Sentiment:** 0.371

## Forecast for AAPL

### AAPL - Combined Sentiment & Forecast Output

,Equity,News,Press,Social,Web,Overall Sentiment,Forecast Next Price
0,AAPL,0.717779,-0.254578,0.110258,0.911313,0.371193,178.301778


In [71]:
# -------------------------------
# Helper Functions
# -------------------------------
def generate_sentiment():
    """Simulate feed-level sentiment scores."""
    feeds = ['News', 'Press', 'Social', 'Web']
    np.random.seed(42)
    feed_scores = {feed: np.random.uniform(-1, 1) for feed in feeds}
    overall_sentiment = np.mean(list(feed_scores.values()))
    return feed_scores, overall_sentiment

def plot_sentiment(equity, feed_scores):
    """Bar chart of sentiment by feed."""
    colors = ['green' if s > 0 else 'red' if s < 0 else 'gray' for s in feed_scores.values()]
    fig = go.Figure(
        data=[go.Bar(x=list(feed_scores.keys()), y=list(feed_scores.values()), marker=dict(color=colors))]
    )
    fig.update_layout(
        title=f"{equity} - Sentiment Scores by Feed",
        yaxis=dict(title="Sentiment Score [-1,1]", range=[-1, 1]),
        xaxis=dict(title="Feed Sources"),
        template="plotly_white",
        bargap=0.3
    )
    fig.show()

def generate_forecast():
    """Simulate historical + forecast price data."""
    dates = pd.date_range('2025-01-01', periods=30)
    hist_prices = np.linspace(150, 180, 30) + np.random.normal(0,2,30)
    forecast_prices = hist_prices[-1] + np.cumsum(np.random.normal(0, 0.5, 7))
    return dates, hist_prices, forecast_prices

def plot_forecast(equity, dates, hist_prices, forecast_prices):
    """Line chart of historical and forecast prices."""
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=dates, y=hist_prices, mode='lines+markers', name='Historical', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=pd.date_range(dates[-1], periods=len(forecast_prices)+1, freq="D")[1:], 
                             y=forecast_prices, mode='lines+markers', name='Forecast', line=dict(color='red')))
    fig.update_layout(title=f"{equity} - Price Forecast (LSTM)", yaxis_title='Price', xaxis_title='Date')
    fig.show()

def show_combined_table(equity, feed_scores, overall_sentiment, forecast_prices):
    """Table combining sentiment + forecast outputs."""
    df = pd.DataFrame({
        'Equity': [equity],
        'News': [feed_scores['News']],
        'Press': [feed_scores['Press']],
        'Social': [feed_scores['Social']],
        'Web': [feed_scores['Web']],
        'Overall Sentiment': [overall_sentiment],
        'Forecast Next Price': [forecast_prices[-1]]
    })
    display(Markdown(f"### {equity} - Combined Sentiment & Forecast Output"))
    display(df)


In [69]:
# -------------------------------
# Master Callback
# -------------------------------
def on_equity_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        equity = change['new']
        
        # Sentiment
        feed_scores, overall_sentiment = generate_sentiment()
        display(Markdown(f"## Sentiment Analysis for {equity}"))
        plot_sentiment(equity, feed_scores)
        display(Markdown(f"**Overall Sentiment:** {overall_sentiment:.3f}"))
        
        # Forecast
        dates, hist_prices, forecast_prices = generate_forecast()
        display(Markdown(f"## Forecast for {equity}"))
        plot_forecast(equity, dates, hist_prices, forecast_prices)
        
        # Combined table
        show_combined_table(equity, feed_scores, overall_sentiment, forecast_prices)

# Attach observer once
equity_selector.observe(on_equity_change, names='value')

### [Open Aligned Equity Forecasting Pipeline in New Tab](equity_pipeline_aligned.svg){target='_blank'}